# Test old vs new remeshing -> Extent problem

In [ ]:
import rasterio
import rasterio.plot
import rasterio.warp
from rasterio.crs import CRS
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import avaframe.in3Utils.fileHandlerUtils as fU
import avaframe.in2Trans.rasterUtils as IOf
import subprocess
import configparser
import avaframe.in2Trans.rasterUtils as rU
%load_ext autoreload
%autoreload 2
import avaframe.in3Utils.geoTrans as geoTrans

In [ ]:
def getIPZ(z0, xEnd, yEnd, dx):
    meanAlpha = 30.0
    nstepsX = int((xEnd + dx) / dx)
    nstepsY = int((yEnd + dx) / dx)
    xv = np.linspace(0, xEnd, nstepsX)
    yv = np.linspace(0, yEnd, nstepsY)
    nRows = len(yv)
    nCols = len(xv)
    x, y = np.meshgrid(xv, yv)
    zv = np.zeros((nRows, nCols))
    # Set surface elevation from slope and max. elevation
    zv = z0 - np.tan(np.radians(meanAlpha)) * x

    return zv

In [ ]:
def getXCoords(src,data):

    height, width = data.shape #Find the height and width of the array

    #Two arrays with the same shape as the input array/raster, where each value is the x or y index of that cell
    cols, rows = np.meshgrid(np.arange(width), np.arange(height)) 

    #Two arrays with the same shape as the input array/raster, where each value is the x or y coordinate of that cell 
    xs, ys = rasterio.transform.xy(src.transform, rows, cols) 

    #They are actually lists, convert them to arrays
    xcoords = np.array(xs[0])
    ycoords = np.array(ys)

    return xcoords

Generate a header info

In [ ]:
cellSize = 5
nCols = 4
nRows = 5
xllcenter = 0
yllcenter = 0
nodata_value = -9999

newCellSize = 2

# rasterio requires west, north
# rasterio.transform.from_origin(west, north, xsize, ysize)
transform = rasterio.transform.from_origin(xllcenter - cellSize / 2,
                                           (yllcenter - cellSize / 2) + nRows * cellSize,
                                           cellSize,
                                           cellSize)
crs = rasterio.crs.CRS()

headerInfo = {
    "cellsize": cellSize,
    "nrows": nRows,
    "ncols": nCols,
    "nodata_value": nodata_value,
    "xllcenter": xllcenter,
    "yllcenter": yllcenter,
    "driver": "AAIGrid",
    "crs": crs,
    "transform": transform,
}
print(headerInfo)
print(transform)

In [ ]:
# create an inclined plane
z0 = 10
data = getIPZ(z0, (nCols -1) * cellSize, (nRows-1)*cellSize, cellSize)

## Old remesh


Generate a flat plane example

In [ ]:

avaDir = pathlib.Path('./', "avaTest")
fU.makeADir(avaDir)
fU.makeADir((avaDir / "Inputs"))
avaDEM = avaDir / "Inputs" / "avaAlr"
avaDEM = IOf.writeResultToRaster(headerInfo, data, avaDEM, flip=True)
subprocess.run(["head", "-n7", avaDEM])

In [ ]:
cfg = configparser.ConfigParser()
cfg["GENERAL"] = {
    "meshCellSizeThreshold": "0.0001",
    "meshCellSize": newCellSize,
    "avalancheDir": str(avaDir),
}

# call function
# %timeit geoTrans.remeshRaster(avaDEM, cfg, legacy=True)
pathDem = geoTrans.remeshRaster(avaDEM, cfg, legacy=True)
fullP = avaDir / "Inputs" / pathDem
print(fullP)
subprocess.run(["head", "-n7", fullP])

So llcenter stays the same at 0,0 and western column keeps the same values, but eastern column is inside the original
 grid (i.e. higher value than the original one

## New remesh

In [ ]:
avaDirNew = pathlib.Path('./', "avaTestNew")
fU.makeADir(avaDirNew)
fU.makeADir((avaDirNew / "Inputs"))
avaDEMNew = avaDirNew / "Inputs" / "avaAlr"
avaDEMNew = IOf.writeResultToRaster(headerInfo, data, avaDEMNew, flip=True)


In [ ]:
cfg = configparser.ConfigParser()
cfg["GENERAL"] = {
    "meshCellSizeThreshold": "0.0001",
    "meshCellSize": newCellSize,
    "avalancheDir": str(avaDirNew),
}

#%timeit geoTrans.remeshRaster(avaDEMNew, cfg)
pathDemNew = geoTrans.remeshRaster(avaDEMNew, cfg)
fullPNew = avaDirNew / "Inputs" / pathDem
subprocess.run(["head", "-n7", fullPNew])

So llcenter moved to -1.5,-0.5 and 2 columns more (1 east 1 west) than legacy and one row more

## Plotting the differences

In [ ]:
oldRaster = rU.readRaster(fullP)
newRaster = rU.readRaster(fullPNew)
#print(oldRaster["header"])
#print(newRaster["header"])
subprocess.run(["head", "-n7", fullP])
subprocess.run(["head", "-n7", fullPNew])

In [ ]:
rasterio.plot.show(oldRaster['rasterData'])
rasterio.plot.show(newRaster['rasterData'])

In [ ]:
srcOrigDEM = rasterio.open(avaDEM, 'r')
dataOrigDEM = srcOrigDEM.read(1)
origX = getXCoords(srcOrigDEM, dataOrigDEM)

srcOrigRemesh = rasterio.open(fullP, 'r')
dataOrigRemesh = srcOrigRemesh.read(1)
origRemeshX = getXCoords(srcOrigRemesh,dataOrigRemesh)

srcNewRemesh = rasterio.open(fullPNew, 'r')
dataNewRemesh = srcNewRemesh.read(1)
dataNewRemesh = np.where(dataNewRemesh == srcNewRemesh.nodata, np.nan, dataNewRemesh)
newX = getXCoords(srcNewRemesh,dataNewRemesh)



# Plot the GeoTIFF
#fig, ax = plt.subplots(figsize=(10, 10))
fig, (axorig, axOrigRemesh,axNew) = plt.subplots(1, 3, figsize=(19,8))
axorig.set_title("OrigDEM")
axorig.set_xlim([-5, 20])
axorig.set_ylim([-5, 25])
axorig.hlines([-2.5,22.5],-5,20)
axorig.vlines([-2.5,17.5],-5,25)
axorig.grid()
axOrigRemesh.set_title("Orig Remesh")
axOrigRemesh.set_xlim([-5, 20])
axOrigRemesh.set_ylim([-5, 25])
axOrigRemesh.hlines([-2.5,22.5],-5,20)
axOrigRemesh.vlines([-2.5,17.5],-5,25)
axOrigRemesh.grid()
axNew.set_title("New Remesh")
axNew.set_xlim([-5, 20])
axNew.set_ylim([-5, 25])
axNew.hlines([-2.5,22.5],-5,20)
axNew.vlines([-2.5,17.5],-5,25)
axNew.grid()

rasterio.plot.show(dataOrigDEM, transform=srcOrigDEM.transform, ax=axorig, alpha=1, cmap="viridis", zorder=5, vmin=0, vmax=10)
rasterio.plot.show(dataOrigRemesh, transform=srcOrigRemesh.transform, ax=axOrigRemesh, alpha=1, cmap="viridis", zorder=5, vmin=0, vmax=10)
rasterio.plot.show(dataNewRemesh, transform=srcNewRemesh.transform, ax=axNew, alpha=1, cmap="viridis", zorder=5, vmin=0, vmax=10)


In [ ]:
fig2, ax2 = plt.subplots(figsize=(19,8))
profOrig = dataOrigDEM[1,:]
profOldRe = dataOrigRemesh[1,:]
profNewRe = dataNewRemesh[1,:]
print(profNewRe)
plt.plot(origX,profOrig,'-*',label="Orig DEM")
plt.plot(origRemeshX,profOldRe,'-x',label="Orig Remesh")
plt.plot(newX,profNewRe,'-X',label='New Remesh')
ax2.legend()